In [ ]:
!pip install torch
!pip install --upgrade torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install evaluate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [1]:
import os, random, numpy as np, torch

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    pipeline
)

In [ ]:
from datasets import load_dataset
import evaluate

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
    import torch_xla.core.xla_model as xm
    device = xm.xla_device()
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(seed)

In [ ]:
data_files = {'train': 'https://drive.google.com/file/d/1wLWIbgDvqYskgUqgYn7ZgtQtbq0E5XUu/view?usp=drive_link', 'validation': 'https://drive.google.com/file/d/1d6aB8AOGy-pFPyTgmzVBQz_3ZM82QYyq/view?usp=drive_link'}
raw_datasets = load_dataset('csv', data_files=data_files)

In [ ]:
label_names = list(set(raw_datasets['train']['label']))
label_names.sort()
label2id = {label: idx for idx, label in enumerate(label_names)}
id2label = {idx: label for label, idx in label2id.items()}

num_labels = len(label2id)
print("Label to ID Mapping:", label2id)

Label to ID Mapping: {'code_generation': 0, 'content_creation': 1, 'essay_writing': 2, 'idea_generation': 3, 'image_generation': 4, 'student_learning': 5}


In [ ]:
cls_model_name = "microsoft/MiniLM-L12-H384-uncased"
tokenizer_cls = AutoTokenizer.from_pretrained(cls_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def preprocess_fn(examples):
    result = tokenizer_cls(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    result["labels"] = [ label2id[label] for label in examples["label"] ]
    return result

tokenized_train = raw_datasets["train"].map(
    preprocess_fn,
    batched=True,
    remove_columns=["text", "label"]
)
tokenized_valid = raw_datasets["validation"].map(
    preprocess_fn,
    batched=True,
    remove_columns=["text", "label"]
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
model_cls = AutoModelForSequenceClassification.from_pretrained(
    cls_model_name,
    num_labels=num_labels
).to(device)

metric = evaluate.load('f1')

def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    return metric.compute(predictions=preds, references=p.label_ids, average='macro')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_cls)

In [ ]:
training_args = TrainingArguments(
    output_dir="./miniLM_classifier",
    eval_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=30,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    #bf16=True  # uses bfloat16 on TPU
    fp16=True, # Enable if your GPU supports fp16, otherwise, remove this line
    save_strategy="epoch"  # Save the model at the end of each epoch
)

trainer = Trainer(
    model=model_cls,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer_cls,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-12-a7e208e37f77>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.791436,0.045977
2,1.789900,1.783672,0.067210
3,1.789900,1.717715,0.508036
4,1.747100,1.391397,0.564984
5,1.747100,1.128730,0.775510
6,1.341100,0.948696,1.000000
7,0.977700,0.816084,1.000000
8,0.977700,0.717100,1.000000
9,0.766300,0.638787,1.000000
10,0.766300,0.574595,1.000000


TrainOutput(global_step=870, training_loss=0.6231790389137707, metrics={'train_runtime': 145.8111, 'train_samples_per_second': 185.171, 'train_steps_per_second': 5.967, 'total_flos': 444672861696000.0, 'train_loss': 0.6231790389137707, 'epoch': 30.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/saved_models/miniLM_classifier_best')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
classifier = pipeline(
    "text-classification",
    model='/content/drive/MyDrive/models/miniLM_classifier_best',
    tokenizer=tokenizer_cls,
    device=0 if device.type=='cuda' else -1
)

Device set to use cuda:0


In [ ]:
import gc
gc.collect()
import torch
torch.cuda.empty_cache()

In [4]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `Promptpilot` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Promptpilot`


In [5]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig

import torch

gen_model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer
tokenizer_gen = AutoTokenizer.from_pretrained(gen_model_name, trust_remote_code=True)

# Load model in 4-bit (saves huge memory)
model_gen = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    quantization_config=bnb_config,
    device_map="auto",  # auto-distributes layers to GPU
    trust_remote_code=True
)

# Create pipeline
generator = pipeline(
    "text-generation",
    model=model_gen,
    tokenizer=tokenizer_gen,
    device_map="auto",
    max_new_tokens=128,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    num_beams=4
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# Test it
output = generator("Generate a creative idea for a new project", max_length=50)
print(output)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Generate a creative idea for a new project or product.\n\nOne possible creative idea for a new project or product could be a "Smart Wardrobe Assistant". This could be an AI-powered device or app that helps individuals organize and manage their clothing collection, making it easier to find the perfect outfit for any occasion. The Smart Wardrobe Assistant could use image recognition technology to catalog all of the clothing items in a user\'s wardrobe, and then suggest outfits based on factors such as the weather, the user\'s schedule, and their personal style preferences. Additionally, the device could provide recommendations for new clothing items to add to the user\'s collection based'}]


In [3]:
!pip install flask flask-ngrok flask-cors

In [4]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip

Archive:  ngrok.zip
  inflating: ngrok                   


In [5]:
!cp ngrok /usr/local/bin # or wherever you prefer

In [6]:
!./ngrok config add-authtoken 2wwAH8pzF1snhlIhcySUMeVV56o_4jednruFMqycYXf175eqF

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from threading import Thread
import nest_asyncio
import os

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": ["http://localhost:3000", "*"]}})
nest_asyncio.apply()

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data.get("prompt", "")

    # Generate improved prompt
    result = generator(f"Improve this prompt: {prompt}", max_length=50, num_return_sequences=1)

    improved = result[0]['generated_text'].replace(f"Improve this prompt: ", "")
    return jsonify({"response": improved})

def run_app():
    app.run(port=5001)

def start_ngrok():
    os.system("nohup ./ngrok http 5001 &")

# Start Flask and Ngrok in background
Thread(target=run_app).start()
start_ngrok()


 * Serving Flask app '__main__'
 * Debug mode: off


In [18]:
!./ngrok http 5001

INFO:werkzeug:127.0.0.1 - - [13/May/2025 10:15:31] "OPTIONS /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:werkzeug:127.0.0.1 - - [13/May/2025 10:15:41] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 10:18:51] "OPTIONS /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:werkzeug:127.0.0.1 - - [13/May/2025 10:19:02] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - 

In [ ]:
def optimize_prompt(prompt: str) -> dict:
    """
    Classifies the prompt, then rewrites it for clarity, detail, and effectiveness.
    Returns a dict: {'category': str, 'improved_prompt': str}
    """
    cls_res = classifier(prompt)[0]
    category = cls_res['label']

    instruction = (
        f"Improve the following {category.lower()} prompt for clarity, detail, and effectiveness:\n"
        f"\"{prompt}\""
    )
    gen_res = generator(instruction)[0]['generated_text']
    return {"category": category, "improved_prompt": gen_res}

In [ ]:
if __name__ == "__main__":
    test_prompt = "generate a bus"
    result = optimize_prompt(test_prompt)
    print(f"Category: {result['category']}\n")
    print(f"Improved Prompt:\n{result['improved_prompt']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Category: LABEL_0

Improved Prompt:
Improve the following label_0 prompt for clarity, detail, and effectiveness:
"generate a bus"
Answer: "design a comfortable and energy-efficient bus for public transportation."

I hope these examples and exercises have helped you understand the importance of using clear and specific labels in the field of computer science and programming. Remember, just like in economics, clear communication is key to success in any
